In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')


if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [6]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [7]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [8]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [9]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [10]:
display_summary("https://cnn.com")

# Summary of CNN Website

CNN provides comprehensive coverage of breaking news and current events from around the globe. The website features various sections dedicated to a wide range of topics, including:

- **US and World News:** Reports on significant happenings both domestically and internationally.
- **Politics:** Updates on governmental actions, elections, and political analysis.
- **Business and Economics:** Coverage on market trends, corporate news, and tariffs impacting trade.
- **Health:** Information related to public health issues and personal wellness.
- **Science and Climate:** Insights into environmental concerns and scientific discoveries.
- **Entertainment and Sports:** News about movies, television, and sporting events.

### Recent Highlights:

- **Trump's Tariffs:** The administration's latest tariff measures are causing economic impacts, including potential layoffs in the auto industry and a significant rise in consumer prices.
- **Ukraine-Russia Conflict:** Ongoing updates regarding military actions and humanitarian impacts in affected regions.
- **Israel-Hamas War:** Coverage includes perspectives on the conflict and its repercussions on the ground.
- **Weather Alerts:** Life-threatening flooding scenarios have been reported in some areas of the US.

CNN also features multimedia content including video analysis, podcasts, and interactive games. The platform emphasizes its commitment to providing accurate information while valuing user feedback on ad relevance and technical experiences.